In [2]:
pip install langchain langgraph bitsandbytes peft trl

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Download da base e amostargem

In [2]:
# Clona o repositório
!git clone https://github.com/pubmedqa/pubmedqa.git

# Importa json e exibe uma amostra dos dados
import json

# Caminho para o arquivo carregado
file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print("\n")
print(f"Campos disponíveis no JSON: {list(data[sample_key].keys())}")

print("\n")
print("==================================================")
print("EXPLORAÇÃO DE DADOS MÉDICOS - PUBMEDQA")
print("==================================================")

# Loop pelas 3 primeiras entradas mostrando TODOS os campos solicitados
for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"EXEMPLO #{i+1} | ID (PMID): {key}")
    print("-" * 60)

    # Campos Principais
    print(f"QUESTION (Pergunta): {item.get('QUESTION', 'N/A')}")

    # Tratamento do Contexto (Lista para String)
    contexto_completo = " ".join(item.get('CONTEXTS', []))
    print(f"CONTEXTS (Contexto médico): {contexto_completo[:300]}...")

    # Labels e Decisão
    print(f"LABELS (Rótulos do contexto): {item.get('LABELS', 'N/A')}")
    print(f"FINAL_DECISION (Decisão): {item.get('final_decision', 'N/A')}")
    print(f"LONG_ANSWER (Resposta detalhada): {item.get('LONG_ANSWER', 'N/A')[:300]}...")

    # Metadados e Predições
    print(f"MESHES (Termos médicos): {item.get('MESHES', 'N/A')}")
    print(f"YEAR (Ano de publicação): {item.get('YEAR', 'N/A')}")

    # Campos de Raciocínio (Úteis para análise de erro/validação)
    print(f"REASONING_REQUIRED_PRED: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"REASONING_FREE_PRED: {item.get('reasoning_free_pred', 'N/A')}")

    print("-" * 60)
    print("=" * 64)

print(f"\n✅ Total de registros disponíveis para análise: {len(data)}")



Campos disponíveis no JSON: ['QUESTION', 'CONTEXTS', 'LABELS', 'MESHES', 'YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'final_decision', 'LONG_ANSWER']


EXPLORAÇÃO DE DADOS MÉDICOS - PUBMEDQA
EXEMPLO #1 | ID (PMID): 21645374
------------------------------------------------------------
QUESTION (Pergunta): Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
CONTEXTS (Contexto médico): Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cel...
LABELS (Rótulos do contexto): ['BACKGROUND', 'RESULTS']
FINAL_DECISION (Decisão): yes
LONG_ANSWER (Resposta detalhada): Results depicted mitochondrial dynamics in vivo as PCD progresses within the lace plant, and highlight the correlation of this organell

Cloning into 'pubmedqa'...


# Split de dados

In [12]:
!cd pubmedqa/preprocess && python split_dataset.py pqal

# Pré-processamento de dados médicos

In [ ]:
import json
import re
import unicodedata
import os

def executar_preprocessing(texto):
    if not isinstance(texto, str):
        return ""

    # Normaliza caracteres Unicode (converte símbolos como \u0394, \u03a8 em texto plano compatível)
    texto = unicodedata.normalize('NFKC', texto)

    # Remove espaços extras, quebras de linha e tabs
    texto = re.sub(r'\s+', ' ', texto).strip()

    # Mantém letras, números, espaços e pontuações clínicas essenciais (incluindo %, + e -)
    # Ajustado para não remover símbolos de fórmulas médicas importantes
    texto = re.sub(r'[^\w\s.,?!():%\-\+]', '', texto)

    return texto

def processar_arquivo(input_path, output_path):
    """Processa um único arquivo JSON"""
    # Carrega os dados brutos
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    dados_limpos = {}

    # Itera sobre os registros para padronizá-los
    for key, item in data.items():
        # Preprocessing da Pergunta (QUESTION)
        pergunta_limpa = executar_preprocessing(item.get('QUESTION', ''))

        # Preprocessing do Contexto (Unindo a lista de CONTEXTS em uma string única limpa)
        contexto_bruto = " ".join(item.get('CONTEXTS', []))
        contexto_limpo = executar_preprocessing(contexto_bruto)

        # Unificação e Preprocessing da Resposta (Decisão + Explicação Detalhada)
        decisao = item.get('final_decision', 'N/A').upper()
        resposta_longa = item.get('LONG_ANSWER', '')
        # Criamos uma resposta estruturada para o treinamento
        resposta_unificada = executar_preprocessing(f"Decisão: {decisao}. Justificativa: {resposta_longa}")

        # Armazena os dados limpos
        dados_limpos[key] = {
            "QUESTION": pergunta_limpa,
            "CONTEXTS": contexto_limpo,
            "FINAL_ANSWER": resposta_unificada,
            "YEAR": item.get('YEAR', 'N/A')
        }

    # Salva o resultado do Preprocessing
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(dados_limpos, f_out, indent=4, ensure_ascii=False)

    return len(dados_limpos)

# Criar diretório de saída para dados processados
os.makedirs('data_processed', exist_ok=True)

print("=" * 60)
print("INICIANDO PRÉ-PROCESSAMENTO DOS DADOS SPLIT")
print("=" * 60)

# Processar test_set.json
print("\n[1/11] Processando test_set.json...")
total = processar_arquivo(
    'pubmedqa/data/test_set.json',
    'data_processed/test_set_preprocessed.json'
)
print(f"Concluído: {total} registros processados")

# Processar cada fold (dev_set.json)
for i in range(10):
    print(f"\n[{i+2}/11] Processando fold {i} (dev_set.json)...")
    
    # Criar diretório do fold
    os.makedirs(f'data_processed/pqal_fold{i}', exist_ok=True)
    
    total = processar_arquivo(
        f'pubmedqa/data/pqal_fold{i}/dev_set.json',
        f'data_processed/pqal_fold{i}/dev_set_preprocessed.json'
    )
    print(f"Concluído: {total} registros processados")

print("\n" + "=" * 60)
print("PRÉ-PROCESSAMENTO FINALIZADO COM SUCESSO!")
print("=" * 60)
print(f"Arquivos salvos em: data_processed/")
print(f"  - test_set_preprocessed.json")
print(f"  - pqal_fold0-9/dev_set_preprocessed.json")

INICIANDO PRÉ-PROCESSAMENTO DOS DADOS SPLIT

[1/11] Processando test_set.json...
✅ Concluído: 500 registros processados

[2/11] Processando fold 0 (dev_set.json)...
✅ Concluído: 50 registros processados

[3/11] Processando fold 1 (dev_set.json)...
✅ Concluído: 50 registros processados

[4/11] Processando fold 2 (dev_set.json)...
✅ Concluído: 50 registros processados

[5/11] Processando fold 3 (dev_set.json)...
✅ Concluído: 50 registros processados

[6/11] Processando fold 4 (dev_set.json)...
✅ Concluído: 50 registros processados

[7/11] Processando fold 5 (dev_set.json)...
✅ Concluído: 50 registros processados

[8/11] Processando fold 6 (dev_set.json)...
✅ Concluído: 50 registros processados

[9/11] Processando fold 7 (dev_set.json)...
✅ Concluído: 50 registros processados

[10/11] Processando fold 8 (dev_set.json)...
✅ Concluído: 50 registros processados

[11/11] Processando fold 9 (dev_set.json)...
✅ Concluído: 50 registros processados

PRÉ-PROCESSAMENTO FINALIZADO COM SUCESSO!
Arqui

# Anonimização de dados

In [15]:
import json
import re
import os

def executar_anonimizacao(texto):
    if not isinstance(texto, str):
        return ""

    # Mascara referências a profissionais (Dr, Dra, MD, etc) + Nome
    texto = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME_PROFISSIONAL]', texto)

    # Mascara e-mails
    texto = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[CONTATO_EMAIL]', texto)

    # Mascara localizações geográficas
    locais_regex = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    texto = re.sub(locais_regex, '[LOCALIZACAO_RESTRITA]', texto, flags=re.IGNORECASE)

    # Mascara IDs longos (6+ dígitos)
    texto = re.sub(r'\b\d{6,}\b', '[ID_RESTRITO]', texto)

    # Mascara anos
    texto = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', texto)

    # Mascara URLs
    texto = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL_RESTRITA]', texto)

    return texto

def anonimizar_arquivo(input_path, output_path):
    """Anonimiza um único arquivo JSON"""
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    dados_anonimizados = {}

    for key, item in data.items():
        # Anonimiza o ID
        novo_id = f"HOSP_REG_{key[:4]}"

        dados_anonimizados[novo_id] = {
            "QUESTION": executar_anonimizacao(item.get('QUESTION', '')),
            "CONTEXTS": executar_anonimizacao(item.get('CONTEXTS', '')),
            "FINAL_ANSWER": executar_anonimizacao(item.get('FINAL_ANSWER', '')),
            "ORIGINAL_ID": "[OCULTADO_POR_SEGURANCA]"
        }

    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(dados_anonimizados, f_out, indent=4, ensure_ascii=False)

    return len(dados_anonimizados)

# Criar diretório de saída para dados anonimizados
os.makedirs('data_anonymized', exist_ok=True)

print("=" * 60)
print("INICIANDO ANONIMIZAÇÃO DOS DADOS")
print("=" * 60)

# 1. Anonimizar test_set
print("\n[1/11] Anonimizando test_set_preprocessed.json...")
total = anonimizar_arquivo(
    'data_processed/test_set_preprocessed.json',
    'data_anonymized/test_set_anonymized.json'
)
print(f"Concluído: {total} registros protegidos")

# 2. Anonimizar cada fold
for i in range(10):
    print(f"\n[{i+2}/11] Anonimizando fold {i}...")
    
    # Criar diretório do fold
    os.makedirs(f'data_anonymized/pqal_fold{i}', exist_ok=True)
    
    total = anonimizar_arquivo(
        f'data_processed/pqal_fold{i}/dev_set_preprocessed.json',
        f'data_anonymized/pqal_fold{i}/dev_set_anonymized.json'
    )
    print(f"Concluído: {total} registros protegidos")

print("\n" + "=" * 60)
print("ANONIMIZAÇÃO FINALIZADA COM SUCESSO!")
print("=" * 60)
print(f"Arquivos salvos em: data_anonymized/")
print(f"  - test_set_anonymized.json")
print(f"  - pqal_fold0-9/dev_set_anonymized.json")

# Demonstração de segurança
print("\n" + "=" * 60)
print("AUDITORIA DE SEGURANÇA (EXEMPLO)")
print("=" * 60)
with open('data_anonymized/test_set_anonymized.json', 'r', encoding='utf-8') as f:
    amostra = json.load(f)
    primeira_chave = list(amostra.keys())[0]
    print(f"Novo ID: {primeira_chave}")
    print(f"Question: {amostra[primeira_chave]['QUESTION'][:100]}...")
    print(f"ID Original: {amostra[primeira_chave]['ORIGINAL_ID']}")

INICIANDO ANONIMIZAÇÃO DOS DADOS

[1/11] Anonimizando test_set_preprocessed.json...
Concluído: 435 registros protegidos

[2/11] Anonimizando fold 0...
Concluído: 50 registros protegidos

[3/11] Anonimizando fold 1...
Concluído: 50 registros protegidos

[4/11] Anonimizando fold 2...
Concluído: 49 registros protegidos

[5/11] Anonimizando fold 3...
Concluído: 49 registros protegidos

[6/11] Anonimizando fold 4...
Concluído: 50 registros protegidos

[7/11] Anonimizando fold 5...
Concluído: 50 registros protegidos

[8/11] Anonimizando fold 6...
Concluído: 48 registros protegidos

[9/11] Anonimizando fold 7...
Concluído: 50 registros protegidos

[10/11] Anonimizando fold 8...
Concluído: 50 registros protegidos

[11/11] Anonimizando fold 9...
Concluído: 50 registros protegidos

ANONIMIZAÇÃO FINALIZADA COM SUCESSO!
Arquivos salvos em: data_anonymized/
  - test_set_anonymized.json
  - pqal_fold0-9/dev_set_anonymized.json

AUDITORIA DE SEGURANÇA (EXEMPLO)
Novo ID: HOSP_REG_1237
Question: Is ano

# Curadoria de Dados

In [18]:
import json
import os
from collections import Counter

def analisar_qualidade_dados(data):
    """Analisa qualidade dos dados"""
    problemas = {
        'question_vazia': [],
        'context_vazio': [],
        'answer_vazia': [],
        'answer_muito_curta': [],
        'context_muito_curto': []
    }
    
    for key, item in data.items():
        # Verifica campos vazios
        if not item.get('QUESTION', '').strip():
            problemas['question_vazia'].append(key)
        
        if not item.get('CONTEXTS', '').strip():
            problemas['context_vazio'].append(key)
            
        if not item.get('FINAL_ANSWER', '').strip():
            problemas['answer_vazia'].append(key)
        
        # Verifica tamanho mínimo
        if len(item.get('FINAL_ANSWER', '')) < 50:
            problemas['answer_muito_curta'].append(key)
            
        if len(item.get('CONTEXTS', '')) < 100:
            problemas['context_muito_curto'].append(key)
    
    return problemas

def extrair_decisao(final_answer):
    """Extrai a decisão (YES/NO/MAYBE) da resposta"""
    final_answer_upper = final_answer.upper()
    if 'YES' in final_answer_upper or 'SIM' in final_answer_upper:
        return 'YES'
    elif 'NO' in final_answer_upper or 'NÃO' in final_answer_upper or 'NAO' in final_answer_upper:
        return 'NO'
    elif 'MAYBE' in final_answer_upper or 'TALVEZ' in final_answer_upper:
        return 'MAYBE'
    return 'UNKNOWN'

def balancear_classes(data):
    """Analisa distribuição de classes"""
    distribuicao = Counter()
    
    for key, item in data.items():
        decisao = extrair_decisao(item.get('FINAL_ANSWER', ''))
        distribuicao[decisao] += 1
    
    return distribuicao

print("=" * 60)
print("ANÁLISE DE QUALIDADE DOS DADOS")
print("=" * 60)

# Carregar dados anonimizados do test_set
with open('data_anonymized/test_set_anonymized.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

# Análise de qualidade
print("\n📊 Analisando test_set...")
problemas = analisar_qualidade_dados(test_data)

print("\nProblemas encontrados:")
for tipo, ids in problemas.items():
    if ids:
        print(f"  - {tipo}: {len(ids)} registros")

# Balanceamento de classes
print("\n📈 Distribuição de classes (test_set):")
distribuicao_test = balancear_classes(test_data)
for classe, qtd in distribuicao_test.items():
    porcentagem = (qtd / len(test_data)) * 100
    print(f"  - {classe}: {qtd} ({porcentagem:.1f}%)")

# Analisar todos os folds
print("\n📈 Distribuição de classes (todos os folds):")
all_folds_data = {}
for i in range(10):
    with open(f'data_anonymized/pqal_fold{i}/dev_set_anonymized.json', 'r', encoding='utf-8') as f:
        fold_data = json.load(f)
        all_folds_data.update(fold_data)

distribuicao_folds = balancear_classes(all_folds_data)
for classe, qtd in distribuicao_folds.items():
    porcentagem = (qtd / len(all_folds_data)) * 100
    print(f"  - {classe}: {qtd} ({porcentagem:.1f}%)")

print(f"\n✅ Total de registros analisados:")
print(f"  - Test set: {len(test_data)}")
print(f"  - Dev folds: {len(all_folds_data)}")
print(f"  - Total: {len(test_data) + len(all_folds_data)}")

ANÁLISE DE QUALIDADE DOS DADOS

📊 Analisando test_set...

Problemas encontrados:

📈 Distribuição de classes (test_set):
  - YES: 238 (54.7%)
  - NO: 176 (40.5%)
  - MAYBE: 21 (4.8%)

📈 Distribuição de classes (todos os folds):
  - YES: 244 (56.1%)
  - NO: 176 (40.5%)
  - MAYBE: 15 (3.4%)

✅ Total de registros analisados:
  - Test set: 435
  - Dev folds: 435
  - Total: 870


In [19]:
import json
import os

def validar_consistencia(data, max_amostras=5):
    """Valida consistência entre pergunta, contexto e resposta"""
    print("=" * 60)
    print("VALIDAÇÃO DE CONSISTÊNCIA")
    print("=" * 60)
    
    inconsistencias = []
    
    for key, item in list(data.items())[:max_amostras]:
        question = item.get('QUESTION', '')
        context = item.get('CONTEXTS', '')
        answer = item.get('FINAL_ANSWER', '')
        
        # Verificações simples de consistência
        tem_interrogacao = '?' in question
        context_relevante = len(context) > 50
        answer_fundamentada = len(answer) > 30
        
        if not (tem_interrogacao and context_relevante and answer_fundamentada):
            inconsistencias.append({
                'id': key,
                'tem_interrogacao': tem_interrogacao,
                'context_relevante': context_relevante,
                'answer_fundamentada': answer_fundamentada
            })
        
        # Exibir amostra
        print(f"\n📋 Registro: {key}")
        print(f"Pergunta: {question[:100]}...")
        print(f"Contexto (tamanho): {len(context)} caracteres")
        print(f"Resposta: {answer[:150]}...")
        print("-" * 60)
    
    return inconsistencias

# Validar amostra do test_set
with open('data_anonymized/test_set_anonymized.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

inconsistencias = validar_consistencia(test_data, max_amostras=3)

if inconsistencias:
    print(f"\n⚠️ Encontradas {len(inconsistencias)} inconsistências nas amostras")
else:
    print(f"\n✅ Todas as amostras validadas estão consistentes")

VALIDAÇÃO DE CONSISTÊNCIA

📋 Registro: HOSP_REG_1237
Pergunta: Is anorectal endosonography valuable in dyschesia?...
Contexto (tamanho): 1231 caracteres
Resposta: Decisão: YES. Justificativa: Linear anorectal endosonography demonstrated incomplete or even absent relaxation of the anal sphincter and the m. pubore...
------------------------------------------------------------

📋 Registro: HOSP_REG_2616
Pergunta: Is there a connection between sublingual varices and hypertension?...
Contexto (tamanho): 1667 caracteres
Resposta: Decisão: YES. Justificativa: An association was found between sublingual varices and hypertension. Examining the lateral borders of the tongue is easi...
------------------------------------------------------------

📋 Registro: HOSP_REG_1910
Pergunta: Are home sampling kits for sexually transmitted infections acceptable among men who have sex with me...
Contexto (tamanho): 1224 caracteres
Resposta: Decisão: MAYBE. Justificativa: The widespread acceptability of usin

In [ ]:
import json
import os

def formatar_para_finetuning(data, output_path, formato='conversational'):
    """
    Formata dados para fine-tuning no padrão conversacional
    Compatível com LLaMA, Falcon, Mistral, etc.
    """
    dados_formatados = []
    
    for key, item in data.items():
        question = item.get('QUESTION', '')
        context = item.get('CONTEXTS', '')
        answer = item.get('FINAL_ANSWER', '')
        
        # Formato conversacional (formato Alpaca/ChatML)
        exemplo = {
            "id": key,
            "messages": [
                {
                    "role": "system",
                    "content": "Você é um assistente médico especializado. Responda às perguntas baseando-se nas evidências científicas fornecidas no contexto."
                },
                {
                    "role": "user",
                    "content": f"Contexto médico: {context}\n\nPergunta: {question}"
                },
                {
                    "role": "assistant",
                    "content": answer
                }
            ]
        }
        
        dados_formatados.append(exemplo)
    
    # Salvar em formato JSONL (linha por linha)
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in dados_formatados:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    return len(dados_formatados)

# Criar diretório para dados curados
os.makedirs('data_curated', exist_ok=True)

print("=" * 60)
print("FORMATAÇÃO PARA FINE-TUNING")
print("=" * 60)

# Formatar test_set
print("\n[1/11] Formatando test_set...")
with open('data_anonymized/test_set_anonymized.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

total = formatar_para_finetuning(
    test_data,
    'data_curated/test_set_curated.jsonl'
)
print(f"{total} exemplos formatados")

# Formatar cada fold
for i in range(10):
    print(f"\n[{i+2}/11] Formatando fold {i}...")
    
    os.makedirs(f'data_curated/pqal_fold{i}', exist_ok=True)
    
    with open(f'data_anonymized/pqal_fold{i}/dev_set_anonymized.json', 'r', encoding='utf-8') as f:
        fold_data = json.load(f)
    
    total = formatar_para_finetuning(
        fold_data,
        f'data_curated/pqal_fold{i}/dev_set_curated.jsonl'
    )
    print(f"{total} exemplos formatados")

print("\n" + "=" * 60)
print("FORMATAÇÃO CONCLUÍDA!")
print("=" * 60)
print("Formato: JSONL conversacional (compatível com LLaMA, Falcon, Mistral)")
print("Estrutura: system + user (contexto + pergunta) + assistant (resposta)")
print(f"\nArquivos salvos em: data_curated/")

# Exibir exemplo formatado
print("\n" + "=" * 60)
print("EXEMPLO DE DADO FORMATADO")
print("=" * 60)
with open('data_curated/test_set_curated.jsonl', 'r', encoding='utf-8') as f:
    exemplo = json.loads(f.readline())
    print(json.dumps(exemplo, indent=2, ensure_ascii=False)[:800] + "...")

FORMATAÇÃO PARA FINE-TUNING

[1/11] Formatando test_set...
✅ 435 exemplos formatados

[2/11] Formatando fold 0...
✅ 50 exemplos formatados

[3/11] Formatando fold 1...
✅ 50 exemplos formatados

[4/11] Formatando fold 2...
✅ 49 exemplos formatados

[5/11] Formatando fold 3...
✅ 49 exemplos formatados

[6/11] Formatando fold 4...
✅ 50 exemplos formatados

[7/11] Formatando fold 5...
✅ 50 exemplos formatados

[8/11] Formatando fold 6...
✅ 48 exemplos formatados

[9/11] Formatando fold 7...
✅ 50 exemplos formatados

[10/11] Formatando fold 8...
✅ 50 exemplos formatados

[11/11] Formatando fold 9...
✅ 50 exemplos formatados

FORMATAÇÃO CONCLUÍDA!
Formato: JSONL conversacional (compatível com LLaMA, Falcon, Mistral)
Estrutura: system + user (contexto + pergunta) + assistant (resposta)

Arquivos salvos em: data_curated/

EXEMPLO DE DADO FORMATADO
{
  "id": "HOSP_REG_1237",
  "messages": [
    {
      "role": "system",
      "content": "Você é um assistente médico especializado. Responda às pe